# Funnel TFA multiple Inversions

This notebook performs the inversion using Levenberg-Marquadt's algorithm of total field anomaly (TFA).

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cPickle as pickle
import os

from fatiando import utils
from fatiando.gravmag import polyprism
from fatiando.mesher import PolygonalPrism
from fatiando.vis import mpl, myv
from matplotlib import colors, ticker, cm
from IPython.display import Image as img
from matplotlib.mlab import normpdf

/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/mpl.py:76: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")
/home/leonardo/anaconda2/lib/python2.7/site-packages/fatiando/vis/myv.py:51: UserWarning: This module will be removed in v0.7.
  warnings.warn("This module will be removed in v0.7.")


### Auxiliary functions

In [2]:
import sys
sys.path.insert(0, '../../code')

import mag_polyprism_functions as mfun

In [3]:
from datetime import datetime
today = datetime.today()
# dd/mm/YY/Hh/Mm
d4 = today.strftime("%d-%b-%Y-%Hh:%Mm")

# Input

In [4]:
model_dir = 'model.pickle'
data_dir = 'data.pickle'
grid_dir = 'grid.pickle'

### Importing model parameters

In [5]:
with open(model_dir) as w:
        model = pickle.load(w)

In [6]:
with open(grid_dir) as w:
        grid = pickle.load(w)

In [7]:
with open(data_dir) as w:
        data = pickle.load(w)

In [8]:
xp = grid['x']
yp = grid['y']
zp = grid['z']
dobs = data['tfa_obs']

### Parameters of the initial model

In [9]:
M = 20 # number of vertices per prism
L = 5 # number of prisms
P = L*(M+2) + 1 # number of parameters

# magnetization direction
incs = model['inc']
decs = model['dec']

int_min = 6.
int_max = 11.
intensity = np.linspace(int_min, int_max, 6)

# depth to the top, thickness and radius
z0_min = -50.
z0_max = 200.
z0 = np.linspace(z0_min, z0_max, 6)
dz = 350.
r = 2000.

x0 = 0.
y0 = 0.

# main field
inc, dec = data['main_field']

In [10]:
z0

array([-50.,   0.,  50., 100., 150., 200.])

In [11]:
intensity

array([ 6.,  7.,  8.,  9., 10., 11.])

### Limits

In [12]:
# limits for parameters in meters
rmin = 10.
rmax = 3000.
x0min = -1000.
x0max = 1000.
y0min = -1000.
y0max = 1000.
dzmin = 10.
dzmax = 1000.

mmin, mmax = mfun.build_range_param(M, L, rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax)

### Variation

In [13]:
# variation for derivatives
deltax = 0.01*np.max(1000.)
deltay = 0.01*np.max(1000.)
deltar = 0.01*np.max(rmax)
deltaz = 0.01*np.max(dzmax)

### Outcropping parameters

In [14]:
# outcropping body parameters
m_out = np.zeros(M + 2)
#m_out = model['param_vec'][:M+2]

### Folder to save the results

In [15]:
foldername = ''

In [16]:
if foldername == '':
    mypath = 'l2-tfa-inversion/multiple-'+d4 #default folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)
else:
    mypath = 'l2_tfa_inversion/multiple-'+foldername #defined folder name
    if not os.path.isdir(mypath):
       os.makedirs(mypath)

### Regularization parameters

In [17]:
#lamb = th*0.01 # Marquadt's parameter
lamb = 10.0
dlamb = 10.      # step for Marquadt's parameter

a1 = 1.0e-5  # adjacent radial distances within each prism
a2 = 1.0e-4   # vertically adjacent radial distances
a3 = 0.     # outcropping cross-section
a4 = 0.     # outcropping origin
a5 = 1.0e-4     # vertically adjacent origins
a6 = 1.0e-7   # zero order Tikhonov on adjacent radial distances
a7 = 1.0e-6     # zero order Tikhonov on thickness of each prism

In [18]:
delta = np.array([deltax, deltay, deltar, deltaz])
alpha = np.array([a1, a2, a3, a4, a5, a6, a7])

In [19]:
itmax = 30
itmax_marq = 10
tol = 1.0e-4     # stop criterion

### Inversion

In [20]:
inversion_results = []
for j, z in enumerate(z0):
    for k, i in enumerate(intensity):
        alpha = np.array([a1, a2, a3, a4, a5, a6, a7])
        print 'inversion: %d  top: %d  intensity: %d' % (j*z0.size + k, z, i)
        model0, m0 = mfun.initial_cylinder(M, L, x0, y0, z, dz, r, inc, dec, incs, decs, i)
        d_fit, m_est, model_est, phi_list, model_list, res_list = mfun.levmarq_tf(
            xp, yp, zp, m0, M, L, delta,
            itmax, itmax_marq, lamb,
            dlamb, tol, mmin, mmax,
            m_out, dobs, inc, dec,
            model0[0].props, alpha, z, dz
        )
        inversion_results.append([m_est, phi_list, model_list, dobs - d_fit])
        print phi_list

inversion: 0  top: -50  intensity: 6
19269.198468549406
19272.71058329121
it:  0   it_marq:  0   lambda: 1e+01   init obj.: 4.31821e+04  fin obj.: 1.92727e+04
22024.206290583283
22057.37688292072
it:  1   it_marq:  0   lambda: 1e+00   init obj.: 1.92727e+04  fin obj.: 2.20574e+04
13771.469671621795
13775.558103787116
it:  1   it_marq:  1   lambda: 1e+01   init obj.: 1.92727e+04  fin obj.: 1.37756e+04
21691.47176701724
21707.795000760776
it:  2   it_marq:  0   lambda: 1e+00   init obj.: 1.37756e+04  fin obj.: 2.17078e+04
12288.965212455514
12292.734914568757
it:  2   it_marq:  1   lambda: 1e+01   init obj.: 1.37756e+04  fin obj.: 1.22927e+04
24552.980225643303
24560.44114825509
it:  3   it_marq:  0   lambda: 1e+00   init obj.: 1.22927e+04  fin obj.: 2.45604e+04
12295.252169757061
12299.179979571643
it:  3   it_marq:  1   lambda: 1e+01   init obj.: 1.22927e+04  fin obj.: 1.22992e+04
12236.051940199415
12239.82334906083
it:  3   it_marq:  2   lambda: 1e+02   init obj.: 1.22927e+04  fin ob

# Results

In [21]:
# output of inversion
inversion = dict()

inversion['results'] = inversion_results
inversion['x'] = xp
inversion['y'] = yp
inversion['z'] = zp
inversion['observed_data'] = dobs
inversion['inc_dec'] = [incs, decs]
inversion['z0'] = z0
inversion['initial_dz'] = dz
inversion['intial_r'] = r
inversion['limits'] = [rmin, rmax, x0min, x0max, y0min, y0max, dzmin, dzmax]
inversion['regularization'] = np.array([a1, a2, a3, a4, a5, a6, a7])
inversion['tol'] = tol
inversion['main_field'] = [inc, dec]
inversion['intensity'] = intensity

### Saving the results

In [22]:
file_name = mypath+'/inversion.pickle'
with open(file_name, 'w') as f:
    pickle.dump(inversion, f)